In [ ]:
# 读取数据
from keras.preprocessing.image import ImageDataGenerator

IMSIZE = 224
train_generator = ImageDataGenerator(rescale=1. / 255).flow_from_directory('../../data/data_inception/train',
                                                                           target_size=(IMSIZE, IMSIZE),
                                                                           batch_size=100,
                                                                           class_mode='categorical'
                                                                           )

validation_generator = ImageDataGenerator(rescale=1. / 255).flow_from_directory('../../data/data_inception/test',
                                                                                target_size=(IMSIZE, IMSIZE),
                                                                                batch_size=100,
                                                                                class_mode='categorical')

In [ ]:
# 展示数据
from matplotlib import pyplot as plt

plt.figure()
fig, ax = plt.subplots(2, 5)
fig.set_figheight(7)
fig.set_figwidth(15)
ax = ax.flatten()
X, Y = next(train_generator)
for i in range(10): ax[i].imshow(X[i, :, :,: ])